## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [1]:
!nvidia-smi


Mon Jan 12 17:25:32 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Installation for GPU llama-cpp-python
# uncomment and run the following code in case GPU is being used
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121 -q

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.28 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.3/551.3 MB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00


**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [4]:
# For installing the libraries & downloading models from HF Hub
!pip install huggingface_hub pandas tiktoken pymupdf langchain langchain-community chromadb sentence-transformers numpy -q

**Note**:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in ***this notebook***.

In [5]:
#Libraries for processing dataframes,text
import json,os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Question Answering using LLM

#### Downloading and Loading the model

In [6]:
# Option 1: Download from a public URL (GitHub, S3, etc.)
# Replace the URL below with your file's public URL
!wget -q "https://raw.githubusercontent.com/visubramaniam/AI-RAG-GENAI/main/data/medical_diagnosis_manual.pdf" -O medical_diagnosis_manual.pdf

In [8]:
pdf_loader = PyMuPDFLoader("medical_diagnosis_manual.pdf")

In [9]:
merck = pdf_loader.load()

In [10]:
for i in range(3):
    print(f"Page Number : {i+1}",end="\n")
    print(merck[i].page_content,end="\n")

Page Number : 1
vsubrama@me.com
1QVXKHA3T6
meant for personal use by vsubrama@m
shing the contents in part or full is liable
Page Number : 2
vsubrama@me.com
1QVXKHA3T6
This file is meant for personal use by vsubrama@me.com only.
Sharing or publishing the contents in part or full is liable for legal action.
Page Number : 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ...........................................................................................................................................................................................
53
1 - Nutritional Di

In [13]:
merck[20].page_content

'PO2\noxygen partial pressure (or tension)\nPPD\npurified protein derivative (tubercullin)\nppm\nparts per million\nprn\nas needed\nPT\nprothrombin time\nPTT\npartial thromboplastin time\nq\nevery (only in dosages)\nqid\n4 times a day (only in dosages)\nRA\nrheumatoid arthritis\nRBC\nred blood cell\nRNA\nribonucleic acid\nSaO2\narterial oxygen saturation\nsc\nsubcutaneous\nsec\nsecond\nSI\nInternational System of Units\nSIDS\nsudden infant death syndrome\nSLE\nsystemic lupus erythematosus\nsp\nspecies (when referring to the singular) [eg, Clostridium sp]\nspp\nspecies (when referring to the plural) [eg, Nocardia and Myocardia spp]\nsp gr\nspecific gravity\nsq\nsquare\nSSRI\nselective serotonin reuptake inhibitor\nTB\ntuberculosis\ntid\n3 times a day (only in dosages)\nTNF\ntumor necrosis factor\nTPN\ntotal parenteral nutrition\nTSH\nthyroid-stimulating hormone\nURI\nupper respiratory infection\nUTI\nurinary tract infection\nvs\nversus\nWBC\nwhite blood cell\nWHO\nWorld Health Organizat

In [14]:
len(merck)

4114

In [15]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

In [20]:
from dotenv import load_dotenv
import os
HF_TOKEN = os.getenv('HUGGINGFACE_TOKEN')

In [21]:
model_path = hf_hub_download(
    token = HF_TOKEN,
    repo_id=model_name_or_path,
    filename=model_basename
)

mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

In [22]:
#uncomment the below snippet of code if the runtime is connected to GPU.
llm = Llama(
    model_path=model_path,
    n_ctx=2300,
    n_gpu_layers=38,
    n_batch=512
)

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    yes
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
llama_model_load_from_file_impl: using device CUDA0 (Tesla T4) - 14992 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                  

#### Response

In [27]:
def response(query,max_tokens=512,temperature=0,top_p=0.95,top_k=50):
    model_output = llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [28]:
user_input = "What is the protocol for managing sepsis in a critical care unit?"
respstr = response(user_input)

# Display the response as formatted markdown
from IPython.display import display, Markdown
display(Markdown(f"**Response:**\n\n{respstr}"))

Llama.generate: 2 prefix-match hit, remaining 14 prompt tokens to eval
llama_perf_context_print:        load time =     256.16 ms
llama_perf_context_print: prompt eval time =      77.55 ms /    14 tokens (    5.54 ms per token,   180.53 tokens per second)
llama_perf_context_print:        eval time =   22207.58 ms /   511 runs   (   43.46 ms per token,    23.01 tokens per second)
llama_perf_context_print:       total time =   22684.82 ms /   525 tokens
llama_perf_context_print:    graphs reused =        494


**Response:**



Sepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are the general steps for managing sepsis in a critical care unit:

1. Early recognition and suspicion: Septic patients may present with non-specific symptoms such as fever, chills, tachycardia, tachypnea, altered mental status, and lactic acidosis. It is essential to have a high index of suspicion for sepsis, especially in patients with known infections or risk factors.
2. Initial assessment and resuscitation: The first step in managing sepsis is to assess and resuscitate the patient. This includes assessing airway, breathing, circulation, and disability (ABCD) and providing appropriate interventions such as oxygen therapy, fluid resuscitation, and vasopressor support as needed.
3. Source control: Identifying and addressing the source of infection is crucial in managing sepsis. This may involve surgical intervention, such as drainage of an abscess or debridement of necrotic tissue.
4. Antimicrobial therapy: Broad-spectrum antimicrobial therapy should be initiated as soon as possible in suspected sepsis cases. The choice of antibiotics depends on the suspected source of infection and local antibiogram data.
5. Fluid management: Fluid resuscitation is a critical component of sepsis management. Crystalloid solutions are the initial fluid of choice, and colloids may be used in cases of refractory shock.
6. Vasopressor support: In cases of septic shock, vasopressor support may be required to maintain adequate blood pressure and organ perfusion.
7. Corticosteroids: The use of corticosteroids in sepsis remains controversial. Some studies suggest that they may improve outcomes in certain subgroups of patients, such as those with refractory shock or ARDS.
8. Inotropes: Inotropes may be required to support cardiac output in cases of sepsis-induced cardiomyopathy.
9. Renal replacement therapy: In cases of severe sepsis or septic shock with renal dysfunction, renal replacement therapy may be necessary to maintain fluid and electrolyte balance and remove toxins.
1

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [29]:
user_input = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
respstr = response(user_input)

# Display the response as formatted markdown
from IPython.display import display, Markdown
display(Markdown(f"**Response:**\n\n{respstr}"))

Llama.generate: 2 prefix-match hit, remaining 32 prompt tokens to eval
llama_perf_context_print:        load time =     256.16 ms
llama_perf_context_print: prompt eval time =      98.48 ms /    32 tokens (    3.08 ms per token,   324.93 tokens per second)
llama_perf_context_print:        eval time =   19060.05 ms /   398 runs   (   47.89 ms per token,    20.88 tokens per second)
llama_perf_context_print:       total time =   19414.69 ms /   430 tokens
llama_perf_context_print:    graphs reused =        385


**Response:**



Appendicitis is a medical condition characterized by inflammation of the appendix, a small pouch-like structure that extends from the cecum, the first part of the large intestine. The symptoms of appendicitis can vary from person to person, but the following are the most common ones:

1. Abdominal pain: The pain is typically located in the lower right side of the abdomen, although it can also be felt in the middle or left side. The pain may start as a mild discomfort, but it can quickly become severe and constant.
2. Loss of appetite: People with appendicitis may lose their appetite and feel nauseous or vomit.
3. Fever: A fever of 100.4°F (38°C) or higher is common in appendicitis.
4. Abdominal swelling: The abdomen may become swollen and tender to the touch.
5. Diarrhea or constipation: Some people with appendicitis may experience diarrhea, while others may have constipation.

Appendicitis cannot be cured with medicine alone. Antibiotics may be used to treat any accompanying infection, but they will not resolve the inflammation of the appendix. Surgery is the only effective treatment for appendicitis. The most common surgical procedure used to treat appendicitis is an appendectomy, which involves removing the appendix. The appendix is usually removed through a small incision in the abdomen, although laparoscopic surgery may also be used in some cases.

It is important to seek medical attention as soon as possible if you suspect that you have appendicitis. Delaying treatment can increase the risk of complications, such as a ruptured appendix, which can be life-threatening.

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [30]:
user_input = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
respstr = response(user_input)

# Display the response as formatted markdown
from IPython.display import display, Markdown
display(Markdown(f"**Response:**\n\n{respstr}"))

Llama.generate: 4 prefix-match hit, remaining 34 prompt tokens to eval
llama_perf_context_print:        load time =     256.16 ms
llama_perf_context_print: prompt eval time =     107.21 ms /    34 tokens (    3.15 ms per token,   317.13 tokens per second)
llama_perf_context_print:        eval time =   25246.17 ms /   511 runs   (   49.41 ms per token,    20.24 tokens per second)
llama_perf_context_print:       total time =   25752.54 ms /   545 tokens
llama_perf_context_print:    graphs reused =        494


**Response:**



Sudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles, leading to hair loss in small, round patches on the scalp, beard, or other areas of the body. The exact cause of alopecia areata is not known, but it is believed to be related to a problem with the immune system.

There are several treatments that have been shown to be effective in addressing sudden patchy hair loss:

1. Corticosteroids: Corticosteroids are anti-inflammatory medications that can help reduce inflammation and suppress the immune system, allowing the hair follicles to regrow. They can be applied topically or taken orally.
2. Minoxidil: Minoxidil is a medication that has been shown to promote hair growth in some people with alopecia areata. It is applied topically to the affected area.
3. Diphenylcyclopropenone (DPCP): DPCP is a medication that is applied to the scalp to stimulate the immune system and promote hair regrowth. It is used in combination with corticosteroids.
4. Anthralin: Anthralin is a medication that is applied to the scalp to reduce inflammation and promote hair regrowth. It is used in combination with corticosteroids.
5. JAK inhibitors: JAK inhibitors are a new class of medications that have shown promise in treating alopecia areata. They work by suppressing the immune system and promoting hair growth.

The possible causes of sudden patchy hair loss include:

1. Genetics: Alpecia areata is more common in people with a family history of the condition.
2. Stress: Stress can trigger the onset of alopecia areata or worsen existing hair loss.
3. Viruses or infections: Certain viruses or infections may trigger the immune system to attack the hair follicles.
4. Autoimmune disorders: Alpecia areata is an autoimmune disorder, which means that the immune system attacks the body's own tissues.
5. Hormonal imbalances: Hormonal imbalances, such as those caused by thyroid disorders or pregnancy, can lead to sudden patchy hair loss

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [31]:
user_input = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
respstr = response(user_input)

# Display the response as formatted markdown
from IPython.display import display, Markdown
display(Markdown(f"**Response:**\n\n{respstr}"))

Llama.generate: 2 prefix-match hit, remaining 28 prompt tokens to eval
llama_perf_context_print:        load time =     256.16 ms
llama_perf_context_print: prompt eval time =      97.40 ms /    28 tokens (    3.48 ms per token,   287.48 tokens per second)
llama_perf_context_print:        eval time =   24540.37 ms /   508 runs   (   48.31 ms per token,    20.70 tokens per second)
llama_perf_context_print:       total time =   25027.31 ms /   536 tokens
llama_perf_context_print:    graphs reused =        491


**Response:**



A person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function, is typically diagnosed with a traumatic brain injury (TBI). The treatment for a TBI depends on the severity and location of the injury, as well as the individual's overall health and age.

Immediate treatment for a TBI may include:

1. Emergency medical care: This may include surgery to remove hematomas or other obstructions, as well as treatment for any life-threatening conditions, such as airway obstruction or severe bleeding.
2. Medications: Depending on the symptoms, medications may be prescribed to manage conditions such as swelling, pain, or seizures.
3. Rehabilitation: Rehabilitation may include physical therapy, occupational therapy, speech therapy, and cognitive rehabilitation to help the person regain lost skills and functions.
4. Supportive care: This may include assistance with daily living activities, such as bathing, dressing, and feeding, as well as emotional and social support.

Long-term treatment for a TBI may include:

1. Medications: Depending on the symptoms, medications may be prescribed to manage conditions such as seizures, depression, or anxiety.
2. Rehabilitation: Rehabilitation may continue for an extended period to help the person regain as much function as possible.
3. Assistive devices: Assistive devices, such as wheelchairs, walkers, or communication aids, may be necessary to help the person function independently.
4. Vocational rehabilitation: Vocational rehabilitation may be necessary to help the person return to work or find a new job if they are unable to return to their previous employment.
5. Support groups: Joining a support group can provide emotional and social support for the person and their family.
6. Lifestyle modifications: Lifestyle modifications, such as getting enough rest, eating a healthy diet, and engaging in regular physical activity, can help improve overall health and well-being.

It is important to note that every person's recovery from a TBI is unique, and treatment plans may vary depending on the individual's specific needs and circumstances. Regular follow-up care with healthcare professionals is essential to ensure that the person receives the appropriate care and support throughout their recovery process.

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [32]:
user_input = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
respstr = response(user_input)

# Display the response as formatted markdown
from IPython.display import display, Markdown
display(Markdown(f"**Response:**\n\n{respstr}"))

Llama.generate: 2 prefix-match hit, remaining 35 prompt tokens to eval
llama_perf_context_print:        load time =     256.16 ms
llama_perf_context_print: prompt eval time =     107.29 ms /    35 tokens (    3.07 ms per token,   326.23 tokens per second)
llama_perf_context_print:        eval time =   22331.92 ms /   463 runs   (   48.23 ms per token,    20.73 tokens per second)
llama_perf_context_print:       total time =   22778.41 ms /   498 tokens
llama_perf_context_print:    graphs reused =        448


**Response:**



First and foremost, if a person has fractured their leg during a hiking trip, it is essential to ensure their safety and prevent further injury. Here are some necessary precautions and treatment steps:

1. Assess the situation: Check the extent of the injury and assess the person's condition. If the fracture is open or the person is in severe pain, do not move them unless it is necessary to prevent further harm.
2. Call for help: If possible, call for emergency medical assistance. If there is no cell phone reception, try to find a way to signal for help, such as using a mirror to reflect sunlight or using a whistle to attract attention.
3. Provide first aid: If the person's condition is stable, provide first aid by immobilizing the leg with a splint or a makeshift sling. Use a clean cloth to apply pressure to any bleeding wounds and elevate the leg to reduce swelling.
4. Keep the person warm: Hypothermia can be a significant risk in cold environments, so keep the person warm by covering them with blankets or insulating materials.
5. Provide fluids: Dehydration can also be a concern, so provide the person with water or other fluids to keep them hydrated.
6. Transport the person: Once medical assistance arrives, help transport the person to the hospital for further evaluation and treatment.

During the person's care and recovery, consider the following factors:

1. Medical treatment: The person will likely need to undergo surgery to repair the fracture and may require ongoing medical care, such as physical therapy or pain management.
2. Rehabilitation: The person will need to undergo a rehabilitation program to regain strength and mobility in the affected leg.
3. Emotional support: The person may experience anxiety or depression during their recovery, so it is essential to provide emotional support and encouragement.
4. Return to hiking: Once the person has fully recovered, they may need to take precautions to prevent future injuries while hiking, such as wearing proper footwear and taking breaks to rest and hydrate.

## Question Answering using LLM with Prompt Engineering

In [ ]:
system_prompt = """
You are a highly specialized medical information assistant with expertise in interpreting clinical references from the Merck Manual. Your role is to provide accurate, evidence-based medical information to healthcare professionals.

### Instructions:
1. **Context Source**: You will receive context from the Merck Manual, a trusted medical reference covering disorders, diagnostics, treatments, and pharmaceutical information. This context begins with the token: ###Context.

2. **Question Format**: User questions will begin with the token: ###Question.

3. **Response Guidelines**:
   - Provide precise, clinically accurate answers based ONLY on the provided context
   - Use proper medical terminology while maintaining clarity
   - Structure your response with clear sections when appropriate (e.g., Symptoms, Diagnosis, Treatment, Prognosis)
   - Include relevant dosages, procedures, or protocols when mentioned in the context
   - Distinguish between first-line and alternative treatments when applicable

4. **Accuracy Requirements**:
   - Do NOT hallucinate or infer information not present in the context
   - Do NOT provide personal medical advice or diagnoses
   - If the context contains partial information, clearly state what is available and what is missing
   - If the answer is not found in the context, respond: "The provided Merck Manual excerpt does not contain sufficient information to answer this question."

5. **Medical Disclaimer**: Always remember that responses are for informational purposes and should be verified by qualified healthcare professionals before clinical application.

Respond in a clear, professional manner suitable for healthcare practitioners.
"""

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [33]:
user_input = " What is the protocol for managing sepsis in a critical care unit?"
respstr = response(user_input)

# Display the response as formatted markdown
from IPython.display import display, Markdown
display(Markdown(f"**Response:**\n\n{respstr}"))

Llama.generate: 1 prefix-match hit, remaining 16 prompt tokens to eval
llama_perf_context_print:        load time =     256.16 ms
llama_perf_context_print: prompt eval time =      78.06 ms /    16 tokens (    4.88 ms per token,   204.98 tokens per second)
llama_perf_context_print:        eval time =   25249.20 ms /   511 runs   (   49.41 ms per token,    20.24 tokens per second)
llama_perf_context_print:       total time =   25717.27 ms /   527 tokens
llama_perf_context_print:    graphs reused =        494


**Response:**



Sepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are the key steps in managing sepsis in a critical care unit:

1. Early recognition: Recognize sepsis early by identifying the signs and symptoms of infection and sepsis, such as fever, chills, tachycardia, tachypnea, altered mental status, and lactic acidosis. Use validated scoring systems, such as the Sequential Organ Failure Assessment (SOFA) score or the Quick Sequential Organ Failure Assessment (qSOFA) score, to help identify patients who may have sepsis.
2. Fluid resuscitation: Administer intravenous fluids to maintain adequate tissue perfusion and prevent hypotension. The goal is to achieve a mean arterial pressure (MAP) of at least 65 mmHg and a central venous pressure (CVP) of 8-12 mmHg. Use crystalloids initially, and consider colloids or blood products if the patient does not respond to crystalloids.
3. Antibiotics: Administer broad-spectrum antibiotics as soon as possible, based on the suspected source of infection and local antibiogram data. Consider obtaining cultures before administering antibiotics if the source of infection is uncertain.
4. Source control: Identify and address the source of infection as soon as possible. This may involve surgical intervention, such as drainage of an abscess or debridement of necrotic tissue.
5. Vasopressors: If the patient remains hypotensive despite fluid resuscitation, administer vasopressors to maintain adequate tissue perfusion. Commonly used vasopressors include norepinephrine, phenylephrine, and dopamine.
6. Corticosteroids: Consider administering corticosteroids in certain situations, such as septic shock, refractory hypotension, or severe acute respiratory distress syndrome (ARDS).
7. Inotropes: Consider administering inotropes, such as dobutamine or milrinone, to improve cardiac output and tissue perfusion.
8. Mechanical ventilation: Provide mechanical ventilation if the patient develops respiratory

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

## Data Preparation for RAG

### Loading the Data

### Data Overview

#### Checking the first 5 pages

#### Checking the number of pages

### Data Chunking

### Embedding

### Vector Database

### Retriever

### System and User Prompt Template

### Response Function

In [ ]:
def generate_rag_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=k)
    context_list = [d.page_content for d in relevant_document_chunks]

    # Combine document chunks into a single context
    context_for_query = ". ".join(context_list)

    user_message = qna_user_message_template.replace('{context}', context_for_query)
    user_message = user_message.replace('{question}', user_input)

    prompt = qna_system_message + '\n' + user_message

    # Generate the response
    try:
        response = llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temperature,
                  top_p=top_p,
                  top_k=top_k
                  )

        # Extract and print the model's response
        response = response['choices'][0]['text'].strip()
    except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

    return response

## Question Answering using RAG

### Query 1: What is the protocol for managing sepsis in a critical care unit?

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

### Fine-tuning

## Output Evaluation

Let us now use the LLM-as-a-judge method to check the quality of the RAG system on two parameters - retrieval and generation. We illustrate this evaluation based on the answeres generated to the question from the previous section.

- We are using the same Mistral model for evaluation, so basically here the llm is rating itself on how well he has performed in the task.

In [ ]:
groundedness_rater_system_message  = ""

In [ ]:
relevance_rater_system_message = ""

In [ ]:
user_message_template = ""

In [ ]:
def generate_ground_relevance_response(user_input,k=3,max_tokens=128,temperature=0,top_p=0.95,top_k=50):
    global qna_system_message,qna_user_message_template
    # Retrieve relevant document chunks
    relevant_document_chunks = retriever.get_relevant_documents(query=user_input,k=3)
    context_list = [d.page_content for d in relevant_document_chunks]
    context_for_query = ". ".join(context_list)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"""[INST]{qna_system_message}\n
                {'user'}: {qna_user_message_template.format(context=context_for_query, question=user_input)}
                [/INST]"""

    response = llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    answer =  response["choices"][0]["text"]

    # Combine user_prompt and system_message to create the prompt
    groundedness_prompt = f"""[INST]{groundedness_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    # Combine user_prompt and system_message to create the prompt
    relevance_prompt = f"""[INST]{relevance_rater_system_message}\n
                {'user'}: {user_message_template.format(context=context_for_query, question=user_input, answer=answer)}
                [/INST]"""

    response_1 = llm(
            prompt=groundedness_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    response_2 = llm(
            prompt=relevance_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            stop=['INST'],
            )

    return response_1['choices'][0]['text'],response_2['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

### Query 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

## Actionable Insights and Business Recommendations

<font size=6 color='blue'>Power Ahead</font>
___